In [1]:
## import json file
import json
import pandas as pd
import numpy as np

# import both weekly and year files
with open('NFL_data_weekly.json') as json_data:
    data = json.load(json_data)


In [2]:
with open('NFL_data_yearly.json') as json_data:
    data_yearly = json.load(json_data)


In [3]:
## convert the list into dictionary form
d = {}
for i in range(len(data)):
    # key := ('Position', 'year', 'week') format
    key = tuple(list(data[i].keys())[0].split('-'))
    d[key] = list(data[i].values())[0]

d_yearly = {}
for i in range(len(data_yearly)):
    key = tuple(list(data_yearly[i].keys())[0].split('-'))
    d_yearly[key] = list(data_yearly[i].values())[0]

In [4]:
# First, let's find the necessary data for QB, 2011, week 4
key = ('QB', '2011', '4')
titles = d[key][3]
players = d[key][0]
stats = d[key][4]

# make a pandas dataframe
df2 = pd.DataFrame(columns=['pos', 'year', 'week', 'player', 'Fantasy Points']
                  + [t + '_prev' for t in titles] + [t+'_prev_2wk' for t in titles] + [t+'_prev_3wk' for t in titles]
                  + [t + '_prev_1yr' for t in titles])
d_yearly[('QB', '2011', 'all')][2];

In [80]:
for year in range(2011, 2018):
    for week in range(4, 18):
        key = ('QB', str(year), str(week))
        # get statistics types, name of players, and statistics
        titles = d[key][3]
        players = d[key][0]
        stats = d[key][4]
        
        # make a new dataframe 
        df = pd.DataFrame(columns=['pos', 'year', 'week', 'player', 'Fantasy Points']
                          + [t + '_prev' for t in titles] + [t+'_prev_2wk' for t in titles] + [t+'_prev_3wk' for t in titles]
                          + [t + '_prev_1yr' for t in titles])
        
        df['player'] = players
        df['pos'] = list(key)[0]
        df['year'] = list(key)[1]
        df['week'] = list(key)[2]
        df['Fantasy Points'] = stats[0]

        ## extract data of the previous weeks
        # keys for previous 3 weeks
        key_prevs = [tuple([k for k in list(key)[:-1]] + [str(int(key[2])-i)]) for i in range(1, 4)]

        # add data of previous weeks respectively
        for prev in range(len(key_prevs)):
            # extract key
            key_prev = key_prevs[prev]
            players_prev = d[key_prev][0]
            stats_prev = d[key_prev][4]

            # add all information of the previous week
            for idx_prev in range(len(players_prev)):
                # check if the player exists
                if players_prev[idx_prev] in players:
                    idx = players.index(players_prev[idx_prev])
                else:
                    continue
                for idx_col in range(len(stats_prev)):
                    df.iloc[idx, idx_col+5+len(titles)*prev] = stats_prev[idx_col][idx_prev]
        
        ## extract data of the previous year
        key_yearly = ('QB', str(year-1), 'all')
        players_prev_yearly = d_yearly[key_yearly][0]
        stats_prev_yearly = d_yearly[key_yearly][2][1:]
        
        # add all information of the previous year
        for idx_prev in range(len(players_prev_yearly)):
            # check if the player exists
            if players_prev_yearly[idx_prev] in players:
                idx = players.index(players_prev_yearly[idx_prev])
            else:
                continue
        
            for idx_col in range(len(stats_prev_yearly)):
                df.iloc[idx, idx_col+5+len(titles)*len(key_prevs)] = stats_prev_yearly[idx_col][idx_prev]
        
        df2 = df2.append(df)

In [113]:
## save the pandas frame in the csv file
df2 = df2.apply(lambda x: "".join(temp.split(',')) if ',' in x else x, axis=0)
df2.to_csv('QB2.csv', index=False)


In [121]:
## drop the row if it contains more than 2 nan values
x=df2[['Fantasy Points_prev', 'Fantasy Points_prev_2wk', 'Fantasy Points_prev_3wk']]
x = x.astype(float)
x.apply(lambda r: np.nan if r.isnull().sum() >= 2 else r.mean(), axis=1)

0     27.333333
1     18.000000
2     25.666667
3     11.666667
4     18.333333
5     14.000000
6     20.333333
7     22.000000
8     13.000000
9           NaN
10    16.000000
11    35.666667
12    12.000000
13    29.333333
14    17.333333
15    16.666667
16    17.333333
17    11.000000
18     7.666667
19    17.000000
20    31.333333
21     5.500000
22    20.333333
23    12.000000
24    14.333333
25    18.333333
26    27.333333
27    18.666667
28          NaN
29    23.000000
        ...    
8     21.000000
9     15.333333
10          NaN
11    21.333333
12    18.000000
13    11.666667
14    18.000000
15    16.666667
16    19.333333
17    23.000000
18          NaN
19    13.333333
20    18.000000
21    12.000000
22          NaN
23          NaN
24     5.000000
25    16.333333
26    17.000000
27          NaN
28    18.666667
29          NaN
30    12.666667
31    26.333333
32          NaN
33    16.333333
34          NaN
35          NaN
36    14.666667
37          NaN
dtype: float64

In [125]:
## add prev, prev_2wks, prev_3wks momentums
for title in titles:
    temp_1wk = title + '_prev'
    temp_2wk = title + '_prev_2wk'
    temp_3wk = title + '_prev_3wk'
    
    df_temp = df2[[temp_1wk, temp_2wk, temp_3wk]]
    df_temp = df_temp.astype(float)
    df2[title+'_momentum'] = df_temp.apply(lambda r: np.nan if r.isnull().sum() >= 2 else r.mean(), axis=1)

df3 = df2[['pos', 'year', 'week', 'player', 'Fantasy Points'] + [t + '_prev' for t in titles] 
         + [t + '_momentum' for t in titles] + [t + '_prev_1yr' for t in titles]]
# remove ',' from numbers (e.g.: 1,523 -> 1523)
df3['Passing Yards_prev_1yr'] = pd.Series(df3['Passing Yards_prev_1yr']).str.replace(',', '')
df3.to_csv('QB3.csv', index=False)